# CIEQ6232 Public Transport Demand and Network Planning and Operations

### Pre-requisites

In [1]:
%%capture
%run utils.py # Load auxiliary functions

ModuleNotFoundError: No module named 'gtfspy'

In [ ]:
# Show maps inline
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE)

----

# Graph processing

### Parameters
Update the following parameters based on the city you chosed

In [ ]:
path_to_sqlite="chicago.sqlite" # Path where the sqlite database is stored

### Load GTFS data

In [ ]:
g=load_gtfs(path_to_sqlite)

### Generate L-space

In [ ]:
#Get available modes for the city
[mode_to_string(x) for x in g.get_modes()]

In [ ]:
#Generate networkX graph
L=generate_graph(g,
               "Subway", # Chosen mode from previous list
                start_hour=5, # Consider trips from 5am...
                end_hour=24) #  ... to midnight

In [ ]:
plot_graph(L, back_map="OSM")

#### Understanding nodes and edges
Each node has an ID and a dictionary of attributes, including the latitude and longitude (coordinates) and a given name.

An edge is a triplet (n1,n2,attrs), where n1 is the ID of the origin stop, n2 is the ID of the destination stop, and attrs is a dictionary with the attributes of the edge, including:
- duration_avg: the average time in seconds to travel the edge
- d: the length in meters of the edge
- n_vehicles: the total number of vehicles that pass through the edge in the studied period
- route_I_counts: same as n_vehicles but split by line_id

In [ ]:
#An example node
list(L.nodes(data=True))[0]

In [ ]:
#An example edge
list(L.edges(data=True))[0]

### Automatic merge
In this step, all stops with the exact same name are merged if (and only if) they are not farther away than **delta** meters

In [ ]:
L_merged=merge_stops_with_same_name(L,delta=200) #Same name and closer to 200m

In [ ]:
plot_graph(L_merged, back_map="OSM")

### Remove islands 
In this step, disconnected stops are removed from the graph. This should be a rare case, e.g., new stops that are currently being built.

In [ ]:
check_islands(L_merged)

### Merge recommender
In this step possible nodes to merge are recommended and user needs to confirm (Y) or skip (N) the suggestion.
The suggestions are based on a combination of the similarity between the names of the stops (0-100) and on the proximity of the stops

In [ ]:
#### 1st round ####
# Almost overlapping stops (20m) regardless of shared name
string_match=0 
stop_distance=20 
merge_recommender(L_merged,
                    string_match, 
                    stop_distance)

#### 2nd round ####
# Stops with at least 75% of similarity in names and closer to 500m
string_match=75 
stop_distance=500
merge_recommender(L_merged,
                    string_match, 
                    stop_distance)

### Manual merge
Click on two stops (holding shift) and then click on Merge button to merge stops. The last selected stop is merged into the first selected stop.

In [ ]:
manual_merge(L_merged,
            jupyter_url="http://localhost:8888") #Change the port number if you're running in a different port

### Sanity check
This function prints some useful information that might help further cleaning the graph

In [ ]:
sanity_check(L_merged)

### Save L-space graph

In [ ]:
save_graph(L_merged,"chicago.pkl") # Change to desired path

----

# Assignment
Now that you have your cleaned L-space graph....

### Parameters
Update the following parameters based on the city you chosed

In [ ]:
path_to_sqlite="chicago.sqlite" # Path where the sqlite database is stored
L_space_path="chicago.pkl" # Path where the clean L-space graph was stored

### Load GTFS data

In [ ]:
g=load_gtfs(path_to_sqlite)

#Get available modes for the city
[mode_to_string(x) for x in g.get_modes()]

### Load L-space graph

In [ ]:
L_graph=load_graph(L_space_path)

### Create P-space 

In [ ]:
P=P_space(g,
          L_graph,
          start_hour=5, # Same as when building L-space
          end_hour=24, # Same as when building L-space
          mode="Subway") # Same as when building L-space

plot_graph(P,"P")

### Compute shortests paths between all pairs of nodes.
#### Note: this may take several minutes (depending on the size of the graph)
For a given pair of nodes $i$ and $j$, This function computes $sp_1,\ldots, sp_m$, the $m$ shortest paths in $\mathbf{L}$-space, i.e., the shortest in terms of in-vehicle travel time, for each pair of nodes $i$ and $j$

The GTC between pairs of stops accounts for initial and transfer waiting times, in-vehicle travel times, and a time-equivalent penalty cost for transfers.

For each path in L-space, we compute the sum of the waiting times for each leg in the path and the number of transfers, according to the labels and number of hops of the corresponding paths in $\mathbf{P}$-space. 
The GTC is determined by the following weighted sum:

$  \text{GTC}(sp_i)= {\text{in_vehicle_time}(sp_i) + \mathbf{\alpha} \times \text{waiting_time}(sp_i) + \sum_{j=1}^{{\text{n_transfers}}(sp_i)}{\beta_J}}$

In [ ]:
# Waiting penalty (multiplier)
alpha = 2 # Waiting time is multiplied by 2

# Transfer penalties in minutes. Arbitrary size of at least one element, with the minutes to add
# for the first, second, third... transfers. 
betas = [5,15] # 5 minutes for the first transfer, 15 minutes for any transfer after that.

# Number of shortest paths to retrieve
m = 3

gtc = get_all_GTC(L_graph, P, m, alpha, betas)

#### Understanding the results

For each pair of nodes we get as many paths as requested (or as many as available if there are fewer alternative paths).

The results is a dictionary, indexed by the nodes of origin (first index) and the nodes of destination (second index). For each pair of nodes we get a list, with the alternative paths ordered by GTC.

Each path is again modeled as a dictionary, including the following attributes:
- path: the sequence of nodes in the path computed in the L-space graph
- GTC: the GTC corresponding to that path
- in_vehicle: the in-vehicle travel time in minutes of that path
- waiting_time: the waiting-time in minutes of that path (from P-space)
- n_transfers: the number of transfers of that path (hops in P-space)

In [ ]:
#Example: check the GTC between nodes 51 and 37
gtc[51][37]

### Auxiliary functions
Here are some auxiliary functions that might be useful for all your assignments. Use them if you need them.

In [ ]:
#Return the average waiting time (in minutes) per line and direction
average_waiting_time_per_line_per_direction(P)

In [ ]:
#Return the average speed (in km/h) for the whole network
average_speed_network(L_graph)

In [ ]:
#Get dataframe with all events
df=get_events(g,
              mode="Subway",
              start_hour=5,
              end_hour=24)
df.head()

In [ ]:
#Set a new attribute to edges
nx.set_edge_attributes(L_graph, 0, 'capacity') # Set attribute "capacity" with default value 0